In [2]:
import cv2,os,xlrd,xlwt,math
import numpy as np
import matplotlib.pyplot as plt
from skimage.measure import compare_ssim
from skimage.measure.simple_metrics import compare_psnr

import os
import pathlib

import torch
import torchvision.transforms as TF
from PIL import Image
from scipy import linalg
from torch.nn.functional import adaptive_avg_pool2d

from pytorch_fid.inception import InceptionV3

try:
    from tqdm import tqdm
except ImportError:
    # If tqdm is not available, provide a mock version of it
    def tqdm(x):
        return x

In [5]:
class ImagePathDataset(torch.utils.data.Dataset):
    def __init__(self, files, transforms=None):
        self.files = files
        self.transforms = transforms

    def __len__(self):
        return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]
        img = Image.open(path).convert('RGB')
        if self.transforms is not None:
            img = self.transforms(img)
        return img


def get_activations(files, model, batch_size=50, dims=2048, device='cpu',
                    num_workers=1):
    """Calculates the activations of the pool_3 layer for all images.

    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : Batch size of images for the model to process at once.
                     Make sure that the number of samples is a multiple of
                     the batch size, otherwise some samples are ignored. This
                     behavior is retained to match the original FID score
                     implementation.
    -- dims        : Dimensionality of features returned by Inception
    -- device      : Device to run calculations
    -- num_workers : Number of parallel dataloader workers

    Returns:
    -- A numpy array of dimension (num images, dims) that contains the
       activations of the given tensor when feeding inception with the
       query tensor.
    """
    model.eval()

    if batch_size > len(files):
        print(('Warning: batch size is bigger than the data size. '
               'Setting batch size to data size'))
        batch_size = len(files)

    dataset = ImagePathDataset(files, transforms=TF.ToTensor())
    dataloader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch_size,
                                             shuffle=False,
                                             drop_last=False,
                                             num_workers=num_workers)

    pred_arr = np.empty((len(files), dims))

    start_idx = 0

    for batch in tqdm(dataloader):
        batch = batch.to(device)

        with torch.no_grad():
            pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
        if pred.size(2) != 1 or pred.size(3) != 1:
            pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

        pred = pred.squeeze(3).squeeze(2).cpu().numpy()

        pred_arr[start_idx:start_idx + pred.shape[0]] = pred

        start_idx = start_idx + pred.shape[0]

    return pred_arr


def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).

    Stable version by Dougal J. Sutherland.

    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.

    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1)
            + np.trace(sigma2) - 2 * tr_covmean)


def calculate_activation_statistics(files, model, batch_size=50, dims=2048,
                                    device='cpu', num_workers=1):
    """Calculation of the statistics used by the FID.
    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : The images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size
                     depends on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- device      : Device to run calculations
    -- num_workers : Number of parallel dataloader workers

    Returns:
    -- mu    : The mean over samples of the activations of the pool_3 layer of
               the inception model.
    -- sigma : The covariance matrix of the activations of the pool_3 layer of
               the inception model.
    """
    act = get_activations(files, model, batch_size, dims, device, num_workers)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma


def compute_statistics_of_path(path, model, batch_size, dims, device,
                               num_workers=1):
    if path.endswith('.npz'):
        with np.load(path) as f:
            m, s = f['mu'][:], f['sigma'][:]
    else:
        path = pathlib.Path(path)
        files = sorted([file for ext in IMAGE_EXTENSIONS
                       for file in path.glob('*.{}'.format(ext))])
        m, s = calculate_activation_statistics(files, model, batch_size,
                                               dims, device, num_workers)

    return m, s


def calculate_fid_given_paths(model, paths, batch_size, device, dims, num_workers=1):
    """Calculates the FID of two paths"""
    for p in paths:
        if not os.path.exists(p):
            raise RuntimeError('Invalid path: %s' % p)


    m1, s1 = compute_statistics_of_path(paths[0], model, batch_size,
                                        dims, device, num_workers)
    m2, s2 = compute_statistics_of_path(paths[1], model, batch_size,
                                        dims, device, num_workers)
    fid_value = calculate_frechet_distance(m1, s1, m2, s2)

    return fid_value

In [6]:
cal_fid = True
cal_ssim = True
cal_miou = True


# def ca

def cal_index(base):
    index= {}

    baseroot = os.path.join('./results',base)
    res_dirs = os.listdir(baseroot)
    res_dirs.sort()
    sub_root = 'images'# final_inpainting   MyInpainting  DBII inpainting-RES
    # res_roots = os.path.join(baseroot,model_name,'results/')
    # res_roots = 

    print(res_dirs)
    mpsnr = {}
    mssim = {}
    mfid = {}
    for sdir in res_dirs:
        # phase = sdir.split('_')[0]
        mpsnr[sdir] = []
        mssim[sdir] = []
        gt_roots = os.path.join(baseroot,sdir,sub_root,'real_image')
        res_roots = os.path.join(baseroot,sdir,sub_root,'synthesized_image')
        images = os.listdir(res_roots)
        images.sort()
        length = len(images)

        #计算ssim psnr
        for i in range(length):
            fake_name = os.path.join(res_roots, images[i])
            gt_name = os.path.join(gt_roots,images[i])

            gt = cv2.imread(gt_name)
            fake= cv2.imread(fake_name)
        #     psnr1 = psnr(gt,gt)
            psnr = compare_psnr(gt,fake,255)
            ssim = compare_ssim(gt,fake, multichannel=True)
            mpsnr[sdir].append(psnr)
            mssim[sdir].append(ssim)
    #         print(images[i*6], psnr,ssim)

    for sdir in res_dirs:
        print(sdir,np.mean(mpsnr[sdir]),np.mean(mssim[sdir]) )



In [7]:
cal_index('effect-byol-1')

['addtion_val_50', 'addtion_val_latest', 'removal_val_latest']
addtion_val_50 29.21234793264564 0.9572462173032961
addtion_val_latest 29.76340705367348 0.9580176744216582
removal_val_latest 33.67262314792866 0.963225757007677


In [4]:
cal_index('./muti-scale-2/')

['ONOK_70', 'ONOK_75', 'ONOK_latest', 'new-iter_70', 'new-iter_75', 'new-iter_latest', 'test_arti_65', 'test_arti_70', 'test_arti_75', 'test_arti_latest', 'test_noaug_65', 'test_noaug_70', 'test_noaug_75', 'test_noaug_latest']
ONOK_70 nan nan
ONOK_75 15.676179581279781 0.5701445811586014
ONOK_latest 14.00274040705876 0.4346123000967598
new-iter_70 15.679415062909229 0.5691164648867096
new-iter_75 15.473985054646587 0.56167991791929
new-iter_latest 16.029865199160422 0.5795157196516381
test_arti_65 14.896008937483348 0.46592711420805166
test_arti_70 15.417072266118835 0.47606357868027044
test_arti_75 14.017188949311565 0.4393358694739438
test_arti_latest 15.216740611675347 0.47407854960632295
test_noaug_65 21.82867257719011 0.8283042771166512
test_noaug_70 22.426307397035778 0.848370647225577
test_noaug_75 23.03003467777809 0.8658764848208851
test_noaug_latest 22.110769438016177 0.8484625808824339


/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
cal_index('./muti-scale-1/')

['ONOK_70', 'ONOK_75', 'ONOK_latest', 'new-iter_70', 'new-iter_75', 'new-iter_latest', 'test_arti_65', 'test_arti_70', 'test_arti_75', 'test_arti_latest', 'test_noaug_65', 'test_noaug_70', 'test_noaug_75', 'test_noaug_latest']
ONOK_70 13.979836680086292 0.415072975165207
ONOK_75 13.97720258585212 0.4193679796903845
ONOK_latest 13.667804570145416 0.40851186830718256
new-iter_70 15.205494807241003 0.5502196904121305
new-iter_75 15.343345849404495 0.5523675744484139
new-iter_latest 15.153252388261452 0.5381582046500379
test_arti_65 14.990771458169977 0.4611774126595772
test_arti_70 14.30438237708792 0.44235264725951845
test_arti_75 14.345579479149828 0.44865666694971246
test_arti_latest 15.338239182460184 0.47758271737431385
test_noaug_65 20.570343501769475 0.7866504588016032
test_noaug_70 20.888700091837475 0.7981287364937574
test_noaug_75 20.648768333103984 0.7974805210365302
test_noaug_latest 20.89336063211472 0.801260888538059


In [17]:
cal_index('./new-iter-0-2/')

['ONOK_90', 'ONOK_95', 'ONOK_latest', 'new-iter_90', 'new-iter_95', 'new-iter_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest']
ONOK_90 14.391052656013525 0.46819012839233975
ONOK_95 14.082436557994702 0.4546791068679239
ONOK_latest 14.442955306365572 0.4701560893052068
new-iter_90 15.942168042782205 0.5756506854874935
new-iter_95 15.85910166267947 0.5759318434902096
new-iter_latest 15.620118130253655 0.565683859913551
test_arti_85 14.547310843619854 0.485943373808897
test_arti_90 14.7794963957689 0.48952891773693147
test_arti_95 14.846884620624277 0.4963340839572807
test_arti_latest 15.345136361719767 0.5046758508868041
test_noaug_85 22.927858811248182 0.8592470444056298
test_noaug_90 23.206825036435617 0.8651357578389984
test_noaug_95 22.564451841146877 0.852913879536189
test_noaug_latest 23.32342164217071 0.8596681996277269


In [14]:
cal_index('./new-iter-0/')

['ONOK_90', 'ONOK_95', 'ONOK_latest', 'new-iter_85', 'new-iter_90', 'new-iter_95', 'new-iter_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest']
ONOK_90 13.748654643814387 0.41375278572978985
ONOK_95 13.760012176776117 0.4207177334084443
ONOK_latest 13.912377092333665 0.4244441860127066
new-iter_85 15.23500851664844 0.5470412295009367
new-iter_90 15.458820520425121 0.5577899200972538
new-iter_95 15.105479522672232 0.5496841399764776
new-iter_latest 15.46148997737433 0.5555415901644306
test_arti_85 14.872460684394857 0.45244609569148114
test_arti_90 14.10240018340306 0.43392513178502007
test_arti_95 14.48897102759871 0.4387458087344009
test_arti_latest 14.008985648717195 0.44015723998833306
test_noaug_85 21.488148225717957 0.8163251790323208
test_noaug_90 21.29980270669841 0.8197297808554632
test_noaug_95 21.71358269874727 0.8239782777876936
test_noaug_latest 21.656458567955294 0.81975222699

In [9]:
cal_index('./new-iter-3/')


['ONOK_85', 'ONOK_90', 'ONOK_95', 'ONOK_latest', 'iter3_85', 'iter3_90', 'iter3_95', 'iter3_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest']
ONOK_85 13.946168495477512 0.39740381503854155
ONOK_90 13.911934039573405 0.39953961063611165
ONOK_95 14.096364115625166 0.40970658089668693
ONOK_latest 13.726238383731856 0.3922286370973609
iter3_85 15.824253743080561 0.5273899791152961
iter3_90 15.742677748189815 0.5237197364527639
iter3_95 15.695263184740117 0.5211201553045592
iter3_latest 15.54888724915401 0.5149871268881981
test_arti_85 15.122929633936666 0.4329812495402483
test_arti_90 15.252736648173649 0.4432325643875344
test_arti_95 13.771697659801973 0.41008409961531866
test_arti_latest 13.874634111525678 0.40700532068060535
test_noaug_85 21.45963552394822 0.8155078110203319
test_noaug_90 21.277263257893306 0.8150864143007864
test_noaug_95 21.860493573115388 0.8325187785880439
test_noaug_l

In [8]:
cal_index('./new-iter-2/')


['ONOK_85', 'ONOK_90', 'ONOK_95', 'ONOK_latest', 'iter3_85', 'iter3_90', 'iter3_95', 'iter3_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest']
ONOK_85 13.99342235526219 0.40625234795952664
ONOK_90 14.143524102922173 0.4140698928095435
ONOK_95 13.897774736012309 0.403257816859282
ONOK_latest 14.046649025918287 0.4048150095583317
iter3_85 15.518226726936703 0.5080366259409377
iter3_90 15.53628512002743 0.5090330092358557
iter3_95 15.722153255017853 0.5195590035023215
iter3_latest 15.091996494995003 0.4930472641629968
test_arti_85 14.269658970153639 0.42982844916113055
test_arti_90 14.033199147596772 0.4216979696709472
test_arti_95 14.072108521619583 0.4205463802847567
test_arti_latest 14.529503626130646 0.44009338156035366
test_noaug_85 21.678966250991767 0.8371028325245267
test_noaug_90 22.116055512761235 0.8427104914897543
test_noaug_95 22.02205909360479 0.843718582795601
test_noaug_latest

In [4]:
cal_index('./new-iter-1/')


['ONOK_85', 'ONOK_90', 'ONOK_95', 'ONOK_latest', 'iter2_85', 'iter2_90', 'iter2_95', 'iter2_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest']
ONOK_85 13.679016519013354 0.39972585365649044
ONOK_90 13.921122726520771 0.41178439977519593
ONOK_95 13.947560024925478 0.4145544241210794
ONOK_latest 13.85791161407259 0.4088316403797436
iter2_85 15.126938171793109 0.49644409967443304
iter2_90 15.607873231694311 0.5199857788156418
iter2_95 14.91372485280129 0.486127514810489
iter2_latest 15.151501545949953 0.5103561901206766
test_arti_85 14.573520734663033 0.4511990871644489
test_arti_90 14.690736010911872 0.45947264595931847
test_arti_95 14.177552936438468 0.4365117305137651
test_arti_latest 13.96808154192371 0.43694283964453806
test_noaug_85 20.28203963417637 0.791586575666755
test_noaug_90 20.847668318996444 0.8060828450544251
test_noaug_95 20.99974179788448 0.8053357673935734
test_noaug_latest

In [3]:
cal_index('./newbox-iter2/')


['ONOK_latest', 'iter2_latest', 'test_arti_latest', 'test_noaug_latest', 'test_train_latest']
ONOK_latest 14.276807364955587 0.44030127145152975
iter2_latest 15.666795439037621 0.5110961640028783
test_arti_latest 15.20481509558735 0.47807449629240234
test_noaug_latest 21.935776967242422 0.8303651799530593
test_train_latest 25.21892712086666 0.9029478447884696


In [4]:
cal_index('./newbox-iter3/')


['ONOK_85', 'ONOK_90', 'ONOK_95', 'ONOK_latest', 'iter3_85', 'iter3_90', 'iter3_95', 'iter3_latest', 'test_arti_85', 'test_arti_90', 'test_arti_95', 'test_arti_latest', 'test_noaug_85', 'test_noaug_90', 'test_noaug_95', 'test_noaug_latest', 'test_train_85', 'test_train_90', 'test_train_95', 'test_train_latest']
ONOK_85 14.618244994190343 0.4593914684026814
ONOK_90 15.05529664357162 0.4698958746616616
ONOK_95 14.505147228049337 0.4471880910223319
ONOK_latest 14.604289151412472 0.4537145479987089
iter3_85 15.690939477888401 0.5143232340989308
iter3_90 15.13444830954558 0.4842871911013468
iter3_95 16.270783045364734 0.531085319488845
iter3_latest 15.510463425099116 0.49940968217598875
test_arti_85 15.381501642748377 0.48737844951083403
test_arti_90 14.866382262958496 0.4695997433719024
test_arti_95 14.530825687350971 0.45582634670320715
test_arti_latest 15.136226368066588 0.49042541353855357
test_noaug_85 21.75572376585336 0.8382583224728689
test_noaug_90 21.451666095463956 0.831461947929

In [3]:
cal_index('./spade24-2-seg-fix-newbox/')


['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_latest', 'test_noaug_110', 'test_noaug_115', 'test_noaug_120', 'test_noaug_125', 'test_noaug_130', 'test_noaug_135', 'test_noaug_140', 'test_noaug_latest', 'test_train_110', 'test_train_115', 'test_train_120', 'test_train_125', 'test_train_130', 'test_train_135', 'test_train_140', 'test_train_latest']
test_arti_110 15.055708102949067 0.4800216426053632
test_arti_115 15.041084559246825 0.5037676232508853
test_arti_120 14.596839796926012 0.4890775424174483
test_arti_125 15.209152023360062 0.5018828600931677
test_arti_130 15.315571405852802 0.5051387403981723
test_arti_135 15.167133324974243 0.4936808986582396
test_arti_140 15.283031437546224 0.4982561720037243
test_arti_latest 14.871123660971216 0.4817949340861818
test_noaug_110 22.040001011041667 0.8379329745905885
test_noaug_115 22.282921414445216 0.8407743208262778
test_noaug_120 21.855199012535426 0.82872

In [7]:
cal_index('./imageDtest/')


['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_latest', 'test_noaug_110', 'test_noaug_115', 'test_noaug_120', 'test_noaug_125', 'test_noaug_130', 'test_noaug_135', 'test_noaug_140', 'test_noaug_latest', 'test_train_110', 'test_train_115', 'test_train_120', 'test_train_125', 'test_train_130', 'test_train_135', 'test_train_140', 'test_train_latest']
test_arti_110 15.41435097874943 0.4963520139699587
test_arti_115 14.57120878513429 0.4662344265457118
test_arti_120 14.648255261867916 0.4780681079212501
test_arti_125 15.11540893231946 0.48897851266082243
test_arti_130 15.146549172140263 0.49803781634540695
test_arti_135 14.66750284339074 0.4846384754223356
test_arti_140 14.575903649484207 0.4742023177823626
test_arti_latest 15.11027816137201 0.48161029528381993
test_noaug_110 21.98287119134604 0.8525348946060084
test_noaug_115 21.92759582993586 0.8430615482928103
test_noaug_120 22.28360849004593 0.8485814670

In [4]:
cal_index('./spade24-2-seg-fix-noc/')


['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_latest', 'test_noaug_110', 'test_noaug_115', 'test_noaug_120', 'test_noaug_125', 'test_noaug_130', 'test_noaug_135', 'test_noaug_140', 'test_noaug_latest', 'test_train_110', 'test_train_115', 'test_train_120', 'test_train_125', 'test_train_130', 'test_train_135', 'test_train_140', 'test_train_latest']
test_arti_110 15.488806696201701 0.506020784375745
test_arti_115 14.45070474138985 0.4766130867494035
test_arti_120 14.791837224353005 0.48505133172892745
test_arti_125 14.723848648309701 0.46888055717195243
test_arti_130 14.54125667981109 0.47484959356520634
test_arti_135 15.76035671155999 0.5247543381012598
test_arti_140 14.866684094413932 0.4988011026593858
test_arti_latest 14.394932872107068 0.4584823055490432
test_noaug_110 22.254178267747506 0.8384553807121632
test_noaug_115 22.157685819134674 0.8430582714685725
test_noaug_120 22.410541067615117 0.850991

In [3]:
cal_index('./spade24-2-seg-fix/')

['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_145', 'test_arti_150', 'test_arti_latest', 'test_noaug_110', 'test_noaug_115', 'test_noaug_120', 'test_noaug_125', 'test_noaug_130', 'test_noaug_135', 'test_noaug_140', 'test_noaug_145', 'test_noaug_150', 'test_noaug_latest', 'test_train_110', 'test_train_115', 'test_train_120', 'test_train_125', 'test_train_130', 'test_train_135', 'test_train_140', 'test_train_145', 'test_train_150', 'test_train_latest']
test_arti_110 15.190800314291462 0.48746241705182225
test_arti_115 15.280662021846467 0.4884976406503197
test_arti_120 15.235452350026865 0.5000898221619691
test_arti_125 15.051023896546317 0.494669230690485
test_arti_130 15.069164767450262 0.4896972062046328
test_arti_135 15.338454226128803 0.5027528678913376
test_arti_140 14.796036584090185 0.49601021311561
test_arti_145 15.160992599859032 0.5077830111560196
test_arti_150 14.65842470888135 0.488602525829

In [5]:
cal_index('./spade24-v2-seg/')

['test_arti_140', 'test_arti_145', 'test_arti_150', 'test_arti_latest', 'test_noaug_140', 'test_noaug_145', 'test_noaug_150', 'test_noaug_latest', 'test_train_140', 'test_train_145', 'test_train_150', 'test_train_latest']
test_arti_140 15.49922987244183 0.5234804023399183
test_arti_145 15.878128016002359 0.5432047933186569
test_arti_150 14.920968848201797 0.5035000954057952
test_arti_latest 15.392143267896586 0.513231308905909
test_noaug_140 22.38611469982946 0.856561597786468
test_noaug_145 23.754875849585087 0.8831634141278623
test_noaug_150 23.785691834372535 0.8813904001062832
test_noaug_latest 23.122048235671457 0.8636043022777472
test_train_140 24.882880743034878 0.901909902028665
test_train_145 24.595883121349598 0.911285731100614
test_train_150 24.962471061162596 0.8985433762877992
test_train_latest 24.320875025490096 0.8901577475655753


In [4]:
cal_index('./spade24-v-seg/')

['test_arti_150', 'test_arti_latest', 'test_noaug_150', 'test_noaug_latest', 'test_train_150', 'test_train_latest']
test_arti_150 15.231469712020697 0.4939556138264009
test_arti_latest 14.973891340466176 0.4799179825716637
test_noaug_150 21.236864355220572 0.8075517847339151
test_noaug_latest 20.99550160667669 0.812472280592077
test_train_150 21.381020447257693 0.8036284498935092
test_train_latest 22.06021703429777 0.8085233532983939


In [6]:
cal_index('./spade24-v-10/')

['test_arti_150', 'test_arti_latest', 'test_noaug_150', 'test_noaug_latest', 'test_train_150', 'test_train_latest']
test_arti_150 15.476635603343789 0.5007844354248944
test_arti_latest 14.373152708065266 0.46717205787614224
test_noaug_150 21.979292851202548 0.8214567976798591
test_noaug_latest 21.641763959972995 0.8180466402297358
test_train_150 21.528861839543637 0.8596628105501736
test_train_latest 22.514738737027727 0.8576432486681721


In [10]:
cal_index('./spade24-v/')

['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_145', 'test_arti_150', 'test_arti_latest', 'test_noaug_135', 'test_noaug_140', 'test_noaug_150', 'test_noaug_latest', 'test_train_135', 'test_train_140', 'test_train_150', 'test_train_latest']
test_arti_110 15.333425149778131 0.49968948831609367
test_arti_115 14.290638975830081 0.467473302894625
test_arti_120 14.668324536214888 0.49053345654121955
test_arti_125 14.671328229551944 0.4847577886375721
test_arti_130 15.349490196964002 0.5068706317442826
test_arti_135 15.016198631218593 0.5049108406651431
test_arti_140 14.94136687722583 0.49223456717379555
test_arti_145 14.062389704079115 0.4582773990298085
test_arti_150 14.709394815641627 0.4796792668443207
test_arti_latest 15.039169847072158 0.5130005578254729
test_noaug_135 21.53787126063066 0.8298147570942915
test_noaug_140 21.42838704331724 0.8232440533570534
test_noaug_150 21.839198703344486 0.836793794352

In [9]:
cal_index('./pscont-seg/')

['test_arti_230', 'test_arti_240', 'test_arti_latest', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_230', 'test_train_240', 'test_train_latest']
test_arti_230 15.158986583025076 0.48783175981837307
test_arti_240 14.924931849078403 0.4923000856268958
test_arti_latest 14.693591687699154 0.4950743932597667
test_noaug_230 19.309479430012203 0.7500022650503203
test_noaug_240 19.114419788789014 0.7491565135057372
test_noaug_latest 19.49895607479998 0.7614357373677648
test_train_230 21.307370169820707 0.7993569887949903
test_train_240 21.500806828826235 0.842249923162594
test_train_latest 20.469332498035023 0.8207794841406756


In [8]:
cal_index('./pscont-vgg/')

['test_arti_160', 'test_arti_240', 'test_arti_latest', 'test_noaug_160', 'test_noaug_240', 'test_noaug_latest', 'test_train_160', 'test_train_240', 'test_train_latest']
test_arti_160 14.341323033002254 0.5237650570113009
test_arti_240 12.949933820593836 0.43425803250905987
test_arti_latest 13.38748364209671 0.47113509509132595
test_noaug_160 18.305977662920547 0.7209980406992107
test_noaug_240 18.00283831664255 0.7087201202435226
test_noaug_latest 18.170048594276285 0.7088847093058471
test_train_160 20.390362374118347 0.7885228859563165
test_train_240 20.20636004582948 0.8023917792641285
test_train_latest 20.759291144733698 0.8025039314300851


In [11]:
cal_index('./spade24/')

['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_150', 'test_arti_latest', 'test_noaug_150', 'test_noaug_latest', 'test_train_150', 'test_train_latest']
test_arti_110 14.811415386176792 0.48782607457705734
test_arti_115 14.156486258614885 0.4697632791284126
test_arti_120 13.87153899963303 0.4439064208507057
test_arti_125 13.664230641532306 0.4604381384473663
test_arti_130 14.19100208470684 0.47741539892490864
test_arti_135 14.095543944116459 0.46103650448784705
test_arti_140 14.006959329830048 0.44953984654924795
test_arti_150 13.942157564268129 0.4503580244525434
test_arti_latest 13.982134663142112 0.44302418286137374
test_noaug_150 19.46964813745189 0.7575454546063723
test_noaug_latest 19.733734047368156 0.7587630777384089
test_train_150 21.054226407734298 0.7779955059325488
test_train_latest 20.019339255937865 0.7136430562051619


In [4]:
cal_index('./spade/')

['test_arti_110', 'test_arti_115', 'test_arti_120', 'test_arti_125', 'test_arti_130', 'test_arti_135', 'test_arti_140', 'test_arti_150', 'test_arti_latest', 'test_noaug_140', 'test_noaug_150', 'test_noaug_latest', 'test_train_140', 'test_train_150', 'test_train_latest']
test_arti_110 13.272058604358293 0.4092465781830737
test_arti_115 13.305718839683513 0.3965116090543274
test_arti_120 13.294031717662955 0.41056570571623413
test_arti_125 13.06069479950608 0.41502987387425966
test_arti_130 12.618928318269413 0.37826175605223405
test_arti_135 13.684021783187575 0.4470662861810184
test_arti_140 13.242577345667712 0.4204043765588365
test_arti_150 13.27213857393121 0.4177057644174081
test_arti_latest 13.775972507421622 0.44291476397155033
test_noaug_140 18.50388224017383 0.7274537557943647
test_noaug_150 19.337480385813155 0.7485949154017294
test_noaug_latest 18.939494176106113 0.7480952455974222
test_train_140 20.957388023840604 0.7703438598582564
test_train_150 19.892092671739217 0.788215

In [12]:
cal_index('./cons-fix/')

['test_arti_230', 'test_arti_240', 'test_arti_latest', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_230', 'test_train_240', 'test_train_latest']
test_arti_230 14.294443651883581 0.5012000416743415
test_arti_240 13.610323976009226 0.47076768279436415
test_arti_latest 13.700752600350226 0.47941840027616317
test_noaug_230 17.08723406064577 0.6648563993052056
test_noaug_240 16.712138146525213 0.6395024591669155
test_noaug_latest 16.929496441205924 0.6355616167931813
test_train_230 18.15786499343352 0.7201703871619408
test_train_240 17.625804103176012 0.6904110085589585
test_train_latest 18.161540254543304 0.678858034492245


In [3]:
cal_index('./cons/')

['test_arti_230', 'test_noaug_220', 'test_noaug_240', 'test_noaug_latest', 'test_train_220', 'test_train_latest']
test_arti_230 14.326209738480088 0.5346009167535881
test_noaug_220 18.331716386856 0.7539529218703571
test_noaug_240 18.358882216828395 0.7479188296284806
test_noaug_latest 18.544481682555787 0.7589268067417485
test_train_220 20.550784573543176 0.8222375482849353
test_train_latest 20.879542199435896 0.8449774232522171


In [5]:
cal_index('./cons-new/')

['test_arti_240', 'test_arti_latest', 'test_noaug_240', 'test_noaug_latest', 'test_train_240', 'test_train_latest']
test_arti_240 14.209386786836921 0.5101675689089833
test_arti_latest 14.423766741996465 0.505799543699586
test_noaug_240 18.043586544150827 0.7113425221132166
test_noaug_latest 18.245017287677364 0.7239835221375478
test_train_240 20.18788916353273 0.7743728872480213
test_train_latest 18.889371360749113 0.727712824050012


In [13]:
cal_index('./pix2ps-aug/')

['test_arti_220', 'test_arti_230', 'test_arti_240', 'test_arti_latest', 'test_noaug_220', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_220', 'test_train_230', 'test_train_240', 'test_train_latest']
test_arti_220 14.701329441749728 0.49881283076085964
test_arti_230 15.912563436197914 0.5462663684982546
test_arti_240 14.693046052244442 0.4797473152268196
test_arti_latest 15.75029491095041 0.5752924656265391
test_noaug_220 20.452086918933556 0.7948342740087238
test_noaug_230 20.68013860978752 0.802270010551322
test_noaug_240 19.98803804246106 0.7828298655754916
test_noaug_latest 20.262639667497705 0.7864989979746214
test_train_220 18.965652301406827 0.7619169778622096
test_train_230 21.879441835161508 0.8185095022491253
test_train_240 19.576163252565284 0.7618593350500491
test_train_latest 19.617470918244205 0.7838758426510122


In [14]:
cal_index('./pix2ps/')

['test_arti_240', 'test_noaug_240', 'test_train_240']
test_arti_240 14.39284009286901 0.5007290832929235
test_noaug_240 21.149214990531068 0.8268334775650226
test_train_240 23.538792307381787 0.8634664716958287


In [7]:
cal_index('./muticls-nols-crop-augs-inc128/')

['test_arti_110', 'test_arti_120', 'test_arti_140', 'test_arti_160', 'test_arti_180', 'test_arti_200', 'test_arti_latest', 'test_noaug_110', 'test_noaug_120', 'test_noaug_140', 'test_noaug_160', 'test_noaug_180', 'test_noaug_200', 'test_noaug_latest', 'test_train_110', 'test_train_120', 'test_train_140', 'test_train_160', 'test_train_180', 'test_train_200', 'test_train_latest']
test_arti_110 15.95698596012108 0.6377788411506872
test_arti_120 15.84888090637802 0.640948954810797
test_arti_140 16.210544101352195 0.6393679033668705
test_arti_160 16.022998013696853 0.6256046680286118
test_arti_180 16.445577651592174 0.651369768553252
test_arti_200 16.457233765134365 0.6450800036276474
test_arti_latest 16.4142444996134 0.6419373673140497
test_noaug_110 16.944089959862403 0.6573477832536446
test_noaug_120 16.974308726735483 0.6622670743067963
test_noaug_140 17.378997231961375 0.6746901536692869
test_noaug_160 17.32921321719503 0.6728983484758498
test_noaug_180 17.570188777252138 0.68369165833

In [22]:
cal_index('./muticls-lsgan/')

['test_noaug_120', 'test_noaug_140', 'test_noaug_160', 'test_noaug_180', 'test_noaug_200', 'test_noaug_210', 'test_noaug_220', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_120', 'test_train_140', 'test_train_160', 'test_train_180', 'test_train_200', 'test_train_210', 'test_train_220', 'test_train_230', 'test_train_240', 'test_train_latest']
test_noaug_120 15.643194327761876 0.6083292938234945
test_noaug_140 15.998257284978697 0.6372591614258143
test_noaug_160 14.8150505226702 0.6240388372247792
test_noaug_180 16.40204432363462 0.6574796141061932
test_noaug_200 16.993534466278494 0.6783535398178172
test_noaug_210 17.240965200281092 0.6888597243413477
test_noaug_220 17.214560777407048 0.684304131501126
test_noaug_230 17.36614716037738 0.6892016224368047
test_noaug_240 17.360980057713483 0.6904161335605155
test_noaug_latest 17.313952182984572 0.6882092114061025
test_train_120 16.668870275605578 0.6393579162055123
test_train_140 17.04813926823794 0.6567376424124329


In [23]:
cal_index('./muticls-nolsgan/')

['test_noaug_100', 'test_noaug_120', 'test_noaug_140', 'test_noaug_160', 'test_noaug_180', 'test_noaug_200', 'test_noaug_210', 'test_noaug_220', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_100', 'test_train_120', 'test_train_140', 'test_train_160', 'test_train_180', 'test_train_200', 'test_train_210', 'test_train_220', 'test_train_230', 'test_train_240', 'test_train_latest']
test_noaug_100 16.555145202556364 0.6581038420818734
test_noaug_120 17.27526313988369 0.6835415360811262
test_noaug_140 17.71388381370877 0.700433164231931
test_noaug_160 17.798142524779024 0.707790957241154
test_noaug_180 17.87678227388437 0.7085655469514011
test_noaug_200 17.97119229033732 0.7165718172052232
test_noaug_210 18.090726701677106 0.7164275687776575
test_noaug_220 18.09995337713192 0.7207906463610432
test_noaug_230 18.17520242840458 0.7201718872322063
test_noaug_240 18.20654332970132 0.7212261316960659
test_noaug_latest 18.1621324272815 0.7199055438108746
test_train_100 20.9166

In [5]:
cal_index('./muticls-nolsgan-crop/')

['test_arti_240', 'test_noaug_230', 'test_noaug_240', 'test_noaug_latest', 'test_train_230', 'test_train_240', 'test_train_latest']
test_arti_240 16.587212195659408 0.6728333673781605
test_noaug_230 18.733188561232424 0.7453290755693109
test_noaug_240 18.814118707360116 0.7467438659150823
test_noaug_latest 18.777198313096548 0.7456391695024819
test_train_230 18.495449340729266 0.7269094319287147
test_train_240 18.549279805779328 0.7220617004421683
test_train_latest 18.65836238648316 0.7248718720979104


In [34]:
for sdir in res_dirs:
    print(sdir,np.mean(mpsnr[sdir]),np.mean(mssim[sdir]) )

nores-edge1_100 38.17307292397876 0.9929034873055689
nores-edge1_105 38.201355267181334 0.9928925036521447
nores-edge1_110 38.10220455974153 0.9928842347016132
nores-edge1_115 38.272476801233694 0.9931359899827189
nores-edge1_120 38.21158483747362 0.9929869626313964
nores-edge1_90 37.707237627038175 0.9924140289545913
nores-mask_100 37.97605239336713 0.9922630719114942
nores-mask_105 37.83906438730061 0.9925319115889517
nores-mask_110 37.7716601929075 0.9925963309963967
nores-mask_115 37.86215303146828 0.9924060251216786
nores-mask_120 37.99361688795235 0.9926397055052282
nores-mask_90 37.5651762141018 0.9919359445031252


In [31]:
for sdir in res_dirs:
    print(sdir,np.mean(mpsnr[sdir]),np.mean(mssim[sdir]) )

final-dip_100 37.621510867745535 0.9918534134603767
final-dip_105 37.77579215079111 0.9924399092123553
final-dip_110 37.94222746217166 0.9928483420707146
final-dip_115 37.96995177572411 0.9928319012775293
final-dip_90 37.31046163951808 0.9916635960241976
final-edge1_100 38.51112204594697 0.9937519939737681
final-edge1_105 38.44872335931853 0.9933478977875603
final-edge1_110 38.503020234550696 0.9935309118469222
final-edge1_115 38.62938615574137 0.993570067643436
final-edge1_120 38.788876608974526 0.9938316163799445
final-edge1_90 38.402544108186476 0.9931919720686179
final_mask_100 37.8531514487468 0.9922802198938715
final_mask_105 37.85755638633316 0.9924161747575379
final_mask_110 37.987649179614394 0.9925133243325689
final_mask_115 38.06309762597907 0.9927285858358816
final_mask_120 38.03702142534449 0.992667083275767
final_mask_95 37.84054503167617 0.9922994855559344


In [38]:
ns_root='./noise_img'
resi_root = './residual_img/'
for item in gt_list:
    # if True:
        gt_name = os.path.join(gt_root,item)
        ns_name = os.path.join(ns_root,item)
        gt = cv2.imread(gt_name,0)
        ns= cv2.imread(ns_name,0)
        cv2.imwrite(os.path.join(resi_root,item),gt-ns)
        

In [37]:

#cal dip psnr
baseroot = '/media/ubuntu/b5deacbc-6c2c-4997-a27b-c59101792b42/Inpainting/'
model_name = 'dip'# final_inpainting   MyInpainting  DBII inpainting-RES
res_roots = os.path.join(baseroot,model_name,'results/')
res_roots = '/media/ubuntu/b5deacbc-6c2c-4997-a27b-c59101792b42/Inpainting/dataset/InpaintingDataset/test/results'
res_dirs = os.listdir(res_roots)
res_dirs.sort()
print(res_dirs)
sdir = 'dip'
mpsnr[sdir] = []
mssim[sdir] = []
for item in gt_list:
# if True:
    gt_name = os.path.join(gt_root,item)
    res_name = os.path.join(res_roots,item)
    gt = cv2.imread(gt_name,0)
    res= cv2.imread(res_name,0)
#     psnr1 = psnr(gt,gt)
    psnr = compare_psnr(gt,res,255)
    ssim = compare_ssim(gt,res,255)
    mpsnr[sdir].append(psnr)
    mssim[sdir].append(ssim)
#     print(item, psnr,ssim)
print(sdir,np.mean(mpsnr[sdir]),np.mean(mssim[sdir]) )

dip 29.798957942839085 0.9526187013287518
